In [1]:
import torch
import cv2
import numpy as np
from tkinter import filedialog, Tk, Label, Button
from PIL import Image, ImageTk

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', trust_repo=True)

# Define color names and their HSV ranges
COLOR_RANGES = {
    'red': ([0, 50, 50], [10, 255, 255]),
    'blue': ([100, 50, 50], [140, 255, 255]),
    'green': ([40, 50, 50], [80, 255, 255]),
    'white': ([0, 0, 200], [180, 55, 255]),
    'black': ([0, 0, 0], [180, 255, 30]),
    'yellow': ([20, 100, 100], [30, 255, 255]),
    'gray': ([0, 0, 40], [180, 50, 180])
}

def get_dominant_color(roi):
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    max_pixels = 0
    dominant_color = 'unknown'
    for color, (lower, upper) in COLOR_RANGES.items():
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
        count = cv2.countNonZero(mask)
        if count > max_pixels:
            max_pixels = count
            dominant_color = color
    return dominant_color

def detect_objects(image_path):
    image = cv2.imread(image_path)
    results = model(image)
    detections = results.xyxy[0]

    car_count = 0
    person_count = 0

    for *box, conf, cls in detections:
        label = model.names[int(cls)]
        x1, y1, x2, y2 = map(int, box)
        
        if label in ['car', 'truck', 'bus']:
            car_count += 1
            roi = image[y1:y2, x1:x2]
            if roi.size == 0:
                continue
            color = get_dominant_color(roi)
            color_bgr = {
                'red': (0, 0, 255),
                'blue': (255, 0, 0),
                'green': (0, 255, 0),
                'white': (255, 255, 255),
                'black': (0, 0, 0),
                'yellow': (0, 255, 255),
                'gray': (128, 128, 128)
            }.get(color, (0, 255, 0))
            cv2.rectangle(image, (x1, y1), (x2, y2), color_bgr, 2)
            cv2.putText(image, f'{label} - {color}', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_bgr, 2)

        elif label == 'person':
            person_count += 1
            cv2.rectangle(image, (x1, y1), (x2, y2), (255, 255, 255), 2)
            cv2.putText(image, 'Person', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    return image, car_count, person_count

def upload_and_detect():
    file_path = filedialog.askopenfilename()
    if file_path:
        output_img, cars, people = detect_objects(file_path)
        output_img = cv2.resize(output_img, (800, 600))
        cv2.imshow(f'Detected Cars: {cars} | People: {people}', output_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

# GUI setup
root = Tk()
root.title("Car Colour and People Detection")
root.geometry("400x200")

label = Label(root, text="Upload an image to detect car color and people")
label.pack(pady=20)

btn = Button(root, text="Upload Image", command=upload_and_detect)
btn.pack()

root.mainloop()

Using cache found in C:\Users\User/.cache\torch\hub\ultralytics_yolov5_master
C:\Users\User/.cache\torch\hub\ultralytics_yolov5_master\utils\general.py:32: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg
YOLOv5  2025-7-21 Python-3.10.11 torch-2.7.1+cpu CPU

100%|██████████| 14.1M/14.1M [00:11<00:00, 1.31MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\User/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
